In [2]:
import pandas as pd
import numpy as np
from lifelines import KaplanMeierFitter, NelsonAalenFitter
from lifelines.utils import concordance_index
import seaborn as sns
import matplotlib.pyplot as plt
import featuretools as ft

In [17]:
data_dictionary = pd.read_csv('HCT/data_dictionary.csv')
data_dictionary

,variable,description,type,values
0,dri_score,Refined disease risk index,Categorical,['Intermediate' 'High' 'N/A - non-malignant in...
1,psych_disturb,Psychiatric disturbance,Categorical,['Yes' 'No' nan 'Not done']
2,cyto_score,Cytogenetic score,Categorical,['Intermediate' 'Favorable' 'Poor' 'TBD' nan '...
3,diabetes,Diabetes,Categorical,['No' 'Yes' nan 'Not done']
4,hla_match_c_high,Recipient / 1st donor allele level (high resol...,Numerical,NaN
5,hla_high_res_8,Recipient / 1st donor allele-level (high resol...,Numerical,NaN
6,tbi_status,TBI,Categorical,"['No TBI' 'TBI + Cy +- Other' 'TBI +- Other, <..."
7,arrhythmia,Arrhythmia,Categorical,['No' nan 'Yes' 'Not done']
8,hla_low_res_6,Recipient / 1st donor antigen-level (low resol...,Numerical,NaN
9,graft_type,Graft type,Categorical,['Peripheral blood' 'Bone marrow']


In [3]:
train = pd.read_csv('HCT/train.csv', index_col='ID')
train

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,vent_hist,renal_issue,pulm_severe,prim_disease_hct,hla_high_res_6,cmv_status,hla_high_res_10,hla_match_dqb1_high,tce_imm_match,hla_nmdp_6,hla_match_c_low,rituximab,hla_match_drb1_low,hla_match_dqb1_low,prod_type,cyto_score_detail,conditioning_intensity,ethnicity,year_hct,obesity,mrd_hct,in_vivo_tcd,tce_match,hla_match_a_high,hepatic_severe,donor_age,prior_tumor,hla_match_b_low,peptic_ulcer,age_at_hct,hla_match_a_low,gvhd_proph,rheum_issue,sex_match,hla_match_b_high,race_group,comorbidity_score,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,Bone marrow,No,No,No,IEA,6.0,+/+,NaN,2.0,NaN,6.0,2.0,No,2.0,2.0,BM,NaN,NaN,Not Hispanic or Latino,2016,No,NaN,Yes,NaN,2.0,No,NaN,No,2.0,No,9.942,2.0,FKalone,No,M-F,2.0,More than one race,0.0,90.0,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,No,No,No,AML,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,PB,Intermediate,MAC,Not Hispanic or Latino,2008,No,Positive,No,Permissive,2.0,No,72.290,No,2.0,No,43.705,2.0,Other GVHD Prophylaxis,No,F-F,2.0,Asian,3.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,HIS,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,NaN,NaN,Not Hispanic or Latino,2019,No,NaN,Yes,NaN,2.0,No,NaN,No,2.0,No,33.997,2.0,Cyclophosphamide alone,No,F-M,2.0,More than one race,0.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,ALL,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,Intermediate,MAC,Not Hispanic or Latino,2009,No,Positive,No,Permissive,2.0,No,29.230,No,2.0,No,43.245,2.0,FK+ MMF +- others,No,M-M,2.0,White,0.0,90.0,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,No,No,No,MPN,6.0,+/+,10.0,2.0,NaN,5.0,2.0,No,2.0,2.0,PB,NaN,MAC,Hispanic or Latino,2018,No,NaN,Yes,NaN,2.0,No,56.810,No,2.0,No,29.740,2.0,TDEPLETION +- other,No,M-F,2.0,American Indian or Alaska Native,1.0,90.0,No,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,Intermediate - TED AML case <missing cytogenetics,NaN,Favorable,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,No,No,NaN,ALL,6.0,-/-,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,PB,Intermediate,MAC,Not Hispanic or Latino,2018,No,Negative,Yes,Fully matched,2.0,No,24.212,Yes,2.0,No,51.136,2.0,FK+ MTX +- others(not MMF),NaN,M-F,2.0,More than one race,0.0,NaN,NaN,Bi-directional non-permissive,NaN,"N/A, Mel not given",8.0,NaN,2.0,No,10.0,0.0,18.633
28796,High,No,Poor,Yes,1.0,4.0,No TBI,No,5.0,Peripheral blood,No,No,No,AML,3.0,-/+,6.0,2.0,G/G,4.0,1.0,No,2.0,2.0,PB,TBD,RIC,Hispanic or Latino,2017,No,Positive,No,NaN,1.0,No,30.770,No,1.0,No,18.075,2.0,Cyclophosphamide +- others,No,M-F,1.0,Native Hawaiian or other Pacific Islander,3.0,90.0,No,GvH non-permissive,Related,"N/A, Mel not given",6.0,Yes,1.0,Yes,8.0,1.0,4.892
28797,TBD cytogenetics,NaN,Poor,NaN,2.0,8.0,No TBI,NaN,6.0,Peripheral blood,No,NaN,NaN,IPA,6.0,-/+,10.0,2.0,G/G,6.0,2.0,NaN,2.0,2.0,PB,Poor,MAC,Not Hispanic or Latino,2018,No,NaN,No,GvH non-permissive,2.0,No,22.627,No,2.0,NaN,51.005,2.0,FK+ MMF +- others,NaN,M-F,2.0,Native Hawaiian or other Pacific Islander,5.0,90.0,NaN,GvH non-permissive,Unrelate

In [18]:
train['donor_patient_age_ratio'] = train['donor_age'] / train['age_at_hct']

In [19]:
train['total_hla_matches'] = (train['hla_match_a_high'] + train['hla_match_a_low'] +
                              train['hla_match_b_high'] + train['hla_match_b_low'] +
                              train['hla_match_c_high'] + train['hla_match_c_low'] +
                              train['hla_match_drb1_high'] + train['hla_match_drb1_low'] +
                              train['hla_match_dqb1_high'] + train['hla_match_dqb1_low'])

In [76]:
train['health_status'] = (train['comorbidity_score'] + train['karnofsky_score'])

In [57]:
train['severity_score'] = (train['pulm_severe'].replace({'No': -1, 'Not done': 0, 'Yes': 1}) * 3 + train['hepatic_severe'].replace({'No': -1, 'Not done': 0, 'Yes': 1}) * 3 +
                           train['renal_issue'].replace({'No': -1, 'Not done': 0, 'Yes': 1}) * 2 + train['arrhythmia'].replace({'No': -1, 'Not done': 0, 'Yes': 1}) * 2 + 
                           train['prior_tumor'].replace({'No': -1, 'Not done': 0, 'Yes': 1}) * 2 + train['psych_disturb'].replace({'No': -1, 'Not done': 0, 'Yes': 1}) * 1)

In [77]:
train['graft_diabetes'] = train['graft_type'].astype(str) + '_' + train['diabetes'].astype(str)

# 2. Объединение состояния здоровья
train['health_status_str'] = train['renal_issue'].astype(str) + '_' + train['pulm_severe'].astype(str) + '_' + train['arrhythmia'].astype(str)

# 3. Индикатор наличия заболеваний
# train['any_comorbidity'] = (train['diabetes'] | train['renal_issue'] | train['pulm_severe'] | train['arrhythmia']).astype(int)

# 4. Предтрансплантационное состояние
train['pre_transplant_status'] = train['prim_disease_hct'].astype(str) + '_' + train['cmv_status'].astype(str)

# 5. Комбинация пола и расы
train['sex_race_combination'] = train['sex_match'].astype(str) + '_' + train['race_group'].astype(str)

# 6. Состояние трансплантата
train['transplant_outcome'] = train['graft_type'].astype(str) + '_' + train['gvhd_proph'].astype(str)

# 7. Индикатор лечения
# train['treatment_indicator'] = (train['rituximab'] | train['melphalan_dose'] > 0).astype(int)

# 8. Состояние печени
train['hepatic_status'] = train['hepatic_severe'].astype(str) + '_' + train['hepatic_mild'].astype(str)

In [62]:
train['age_at_hct'] * train['obesity'].replace({'No': -1, 'Not done': 0, 'Yes': 1})

ID
0        -9.942
1       -43.705
2       -33.997
3       -43.245
4       -29.740
          ...  
28795   -51.136
28796   -18.075
28797   -51.005
28798       NaN
28799    -1.035
Length: 28800, dtype: float64

In [69]:
train['cmv_status'].value_counts()

cmv_status
+/+    13596
-/+     7081
+/-     4048
-/-     3441
Name: count, dtype: int64

In [56]:
train['psych_disturb'].replace({'No': -1, 'Not done': 0, 'Yes': 1})

ID
0       -1.0
1       -1.0
2       -1.0
3       -1.0
4       -1.0
        ... 
28795    NaN
28796   -1.0
28797    NaN
28798   -1.0
28799   -1.0
Name: psych_disturb, Length: 28800, dtype: float64

In [36]:
train['age_at_hct'].max()

73.726

In [32]:
train['donor_age'].max()

84.8

In [25]:
train

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,vent_hist,renal_issue,pulm_severe,prim_disease_hct,hla_high_res_6,cmv_status,hla_high_res_10,hla_match_dqb1_high,tce_imm_match,hla_nmdp_6,hla_match_c_low,rituximab,hla_match_drb1_low,hla_match_dqb1_low,prod_type,cyto_score_detail,conditioning_intensity,ethnicity,year_hct,obesity,mrd_hct,in_vivo_tcd,tce_match,hla_match_a_high,hepatic_severe,donor_age,prior_tumor,hla_match_b_low,peptic_ulcer,age_at_hct,hla_match_a_low,gvhd_proph,rheum_issue,sex_match,hla_match_b_high,race_group,comorbidity_score,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,donor_patient_age_ratio,total_hla_matches,health_status
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,Bone marrow,No,No,No,IEA,6.0,+/+,NaN,2.0,NaN,6.0,2.0,No,2.0,2.0,BM,NaN,NaN,Not Hispanic or Latino,2016,No,NaN,Yes,NaN,2.0,No,NaN,No,2.0,No,9.942,2.0,FKalone,No,M-F,2.0,More than one race,0.0,90.0,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356,NaN,NaN,90.0
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,No,No,No,AML,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,PB,Intermediate,MAC,Not Hispanic or Latino,2008,No,Positive,No,Permissive,2.0,No,72.290,No,2.0,No,43.705,2.0,Other GVHD Prophylaxis,No,F-F,2.0,Asian,3.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672,1.654044,20.0,93.0
2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,HIS,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,NaN,NaN,Not Hispanic or Latino,2019,No,NaN,Yes,NaN,2.0,No,NaN,No,2.0,No,33.997,2.0,Cyclophosphamide alone,No,F-M,2.0,More than one race,0.0,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793,NaN,20.0,90.0
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,No,No,No,ALL,6.0,+/+,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,BM,Intermediate,MAC,Not Hispanic or Latino,2009,No,Positive,No,Permissive,2.0,No,29.230,No,2.0,No,43.245,2.0,FK+ MMF +- others,No,M-M,2.0,White,0.0,90.0,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349,0.675916,20.0,90.0
4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,No,No,No,MPN,6.0,+/+,10.0,2.0,NaN,5.0,2.0,No,2.0,2.0,PB,NaN,MAC,Hispanic or Latino,2018,No,NaN,Yes,NaN,2.0,No,56.810,No,2.0,No,29.740,2.0,TDEPLETION +- other,No,M-F,2.0,American Indian or Alaska Native,1.0,90.0,No,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223,1.910222,20.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,Intermediate - TED AML case <missing cytogenetics,NaN,Favorable,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,No,No,NaN,ALL,6.0,-/-,10.0,2.0,P/P,6.0,2.0,No,2.0,2.0,PB,Intermediate,MAC,Not Hispanic or Latino,2018,No,Negative,Yes,Fully matched,2.0,No,24.212,Yes,2.0,No,51.136,2.0,FK+ MTX +- others(not MMF),NaN,M-F,2.0,More than one race,0.0,NaN,NaN,Bi-directional non-permissive,NaN,"N/A, Mel not given",8.0,NaN,2.0,No,10.0,0.0,18.633,0.473482,20.0,NaN
28796,High,No,Poor,Yes,1.0,4.0,No TBI,No,5.0,Peripheral blood,No,No,No,AML,3.0,-/+,6.0,2.0,G/G,4.0,1.0,No,2.0,2.0,PB,TBD,RIC,Hispanic or Latino,2017,No,Positive,No,NaN,1.0,No,30.770,No,1.0,No,18.075,2.0,Cyclophosphamide +- others,No,M-F,1.0,Native Hawaiian or other Pacific Islander,3.0,90.0,No,GvH non-permissive,Related,"N/A, Mel not given",6.0,Yes,1.0,Yes,8.0,1.0,4.892,1.702351,14.0,93.0
28797,TBD cytogenetics,NaN,Poor,NaN,2.0,8.0,No TBI,NaN,6.0,Peripheral blood,No,NaN,NaN,IPA,6.0,-/+,10.0,2.0,G/G,6.0,2.0,NaN,2.0,2.0,PB,Poor,MAC,Not Hispanic 

In [15]:
train.columns.values

array(['dri_score', 'psych_disturb', 'cyto_score', 'diabetes',
       'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia',
       'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue',
       'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status',
       'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match',
       'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low',
       'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail',
       'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity',
       'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high',
       'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low',
       'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph',
       'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group',
       'comorbidity_score', 'karnofsky_score', 'hepatic_mild',
       'tce_div_match', 'donor_related', 'melphalan_dose',
       'hla_low_res_8', 'cardiac', 'hla_match_drb1_h

In [16]:
train.select_dtypes(object).columns.values

array(['dri_score', 'psych_disturb', 'cyto_score', 'diabetes',
       'tbi_status', 'arrhythmia', 'graft_type', 'vent_hist',
       'renal_issue', 'pulm_severe', 'prim_disease_hct', 'cmv_status',
       'tce_imm_match', 'rituximab', 'prod_type', 'cyto_score_detail',
       'conditioning_intensity', 'ethnicity', 'obesity', 'mrd_hct',
       'in_vivo_tcd', 'tce_match', 'hepatic_severe', 'prior_tumor',
       'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match',
       'race_group', 'hepatic_mild', 'tce_div_match', 'donor_related',
       'melphalan_dose', 'cardiac', 'pulm_moderate'], dtype=object)

In [4]:
def label_encoding(df: pd.DataFrame):
    df['label'] = 0
    df.loc[df['efs'] == 0, 'label'] = -df.loc[df['efs'] == 0, 'efs_time']
    df.loc[df['efs'] == 1, 'label'] = df.loc[df['efs'] == 1, 'efs_time']
    return df

train = label_encoding(train)

/tmp/ipykernel_782/1923187341.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ -42.356  -19.793 -102.349 ...  -23.157  -52.351  -25.158]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['efs'] == 0, 'label'] = -df.loc[df['efs'] == 0, 'efs_time']


In [5]:
naf = NelsonAalenFitter()
naf.fit(train['efs_time'], train['efs'])
train['naf_label'] = -naf.cumulative_hazard_at_times(train['efs_time']).values
train.loc[train['efs'] == 0, 'naf_label'] -= 0.1

In [6]:
kmf = KaplanMeierFitter()
kmf.fit(train['efs_time'], train['efs'])
train['km_label'] = kmf.survival_function_at_times(train['efs_time']).values
train.loc[train['efs'] == 0, 'km_label'] -= 0.1